In [1]:
# Load All Necessary Packages

import os


from bs4 import BeautifulSoup
import requests
import time
import re

import pandas as pd
import numpy as np

# set options to be headless, ..
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException   

In [2]:
aws_url='https://explore.skillbuilder.aws/learn/external-ecommerce;view=none;redirectURL=?ctldoc-catalog-0=l-_en~field14-_2'

In [3]:
# Initialize chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',options=options)
wd.implicitly_wait(30)

In [4]:
course_name=[]
course_provider=[]
course_url=[]
course_sdesc=[]
course_duration=[]
course_cost=[]

In [5]:
wd.get(aws_url)
bs=BeautifulSoup(wd.page_source, 'html.parser')



In [6]:
df_fun=pd.read_csv('aws_fundamental.csv')
df_inter=pd.read_csv('aws_intermediate.csv')
df_adv=pd.read_csv('aws_advance.csv')

In [8]:
df_fun.head()

,Course Title,Course URL,Duration,Rating
0,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0
1,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,10m 00s,5.0
2,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0
3,Introduction to Robotics on AWS,https://explore.skillbuilder.aws/learn/course/...,30m 00s,5.0
4,Getting Started with Bottlerocket,https://explore.skillbuilder.aws/learn/course/...,30m 00s,5.0


In [9]:
df_adv.head()

,Course Title,Course URL,Duration,Rating
0,Cloud Audit Academy FDW Interactive Modules,https://explore.skillbuilder.aws/learn/course/...,2h 00m,5.0
1,Performance Tuning on Amazon DocumentDB,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0
2,Architecting on AWS - Accelerator - Online Cou...,https://explore.skillbuilder.aws/learn/course/...,4h 00m,5.0
3,AWS Storage Gateway Deep Dive: Amazon FSx File...,https://explore.skillbuilder.aws/learn/course/...,1h 15m,5.0
4,AWS Storage Gateway Deep Dive: Tape Gateway,https://explore.skillbuilder.aws/learn/course/...,1h 15m,5.0


In [10]:
df_inter['Level']='Intermediate'
df_inter.head()

,Course Title,Course URL,Cost,Duration,Rating,Level
0,.NET Workloads on AWS Lambda,https://explore.skillbuilder.aws/learn/course/...,Free,1h 30m,5.0,Intermediate
1,"Amazon EC2 Observability, Monitoring, and Trou...",https://explore.skillbuilder.aws/learn/course/...,Free,2h 00m,5.0,Intermediate
2,Data Modeling for Amazon DocumentDB,https://explore.skillbuilder.aws/learn/course/...,Free,1h 00m,5.0,Intermediate
3,Building Event-Driven Applications With Amazon...,https://explore.skillbuilder.aws/learn/course/...,Requires Subscription,3h 00m,5.0,Intermediate
4,Building Event-Driven Applications With Amazon...,https://explore.skillbuilder.aws/learn/course/...,Free,2h 00m,4.0,Intermediate


In [11]:
df_fun['Cost']='Free'
df_fun['Level']='Fundamental'
df_fun.head()

,Course Title,Course URL,Duration,Rating,Cost,Level
0,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0,Free,Fundamental
1,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,10m 00s,5.0,Free,Fundamental
2,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0,Free,Fundamental
3,Introduction to Robotics on AWS,https://explore.skillbuilder.aws/learn/course/...,30m 00s,5.0,Free,Fundamental
4,Getting Started with Bottlerocket,https://explore.skillbuilder.aws/learn/course/...,30m 00s,5.0,Free,Fundamental


In [12]:
df_adv['Cost']='Free'
df_adv['Level']='Advanced'
df_adv.head()

,Course Title,Course URL,Duration,Rating,Cost,Level
0,Cloud Audit Academy FDW Interactive Modules,https://explore.skillbuilder.aws/learn/course/...,2h 00m,5.0,Free,Advanced
1,Performance Tuning on Amazon DocumentDB,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0,Free,Advanced
2,Architecting on AWS - Accelerator - Online Cou...,https://explore.skillbuilder.aws/learn/course/...,4h 00m,5.0,Free,Advanced
3,AWS Storage Gateway Deep Dive: Amazon FSx File...,https://explore.skillbuilder.aws/learn/course/...,1h 15m,5.0,Free,Advanced
4,AWS Storage Gateway Deep Dive: Tape Gateway,https://explore.skillbuilder.aws/learn/course/...,1h 15m,5.0,Free,Advanced


In [13]:
frames=[df_fun,df_inter,df_adv]
df=pd.concat(frames)

In [14]:
df.shape

(548, 6)

In [15]:
df['Level'].unique()

array(['Fundamental', 'Intermediate', 'Advanced'], dtype=object)

In [16]:
new_url_list=np.array(df['Course URL'])

In [44]:
len(new_url_list)

548

In [46]:
course_desc=[]
for url in new_url_list:
    wd.get(url)
    time.sleep(1)
    bs = BeautifulSoup(wd.page_source, 'html.parser')
    desc_div=bs.find('div',class_='tabs-description')
    desc=''
    if desc_div is not None:
        if desc_div.p is not None :
            if desc_div.p.text !='':
                desc=desc_div.p.text.strip()
                if 'Note:' in desc:
                    desc=''
            elif desc_div.p.find_next_sibling('p') is not None:
                sib=desc_div.p.find_next_sibling('p')
                desc=sib.text.strip()
                if 'Note:' in desc:
                    desc=''
    course_desc.append(desc)
    

In [47]:
len(course_desc)

548

In [48]:
df['Description']=course_desc

In [49]:
df.head()

,Course Title,Course URL,Duration,Rating,Cost,Level,Description
0,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0,Free,Fundamental,
1,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,10m 00s,5.0,Free,Fundamental,This course provides CEOs and presidents a hig...
2,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,1h 00m,5.0,Free,Fundamental,The AWS Mainframe Modernization service helps ...
3,Introduction to Robotics on AWS,https://explore.skillbuilder.aws/learn/course/...,30m 00s,5.0,Free,Fundamental,The robotics industry is growing at a rapid ra...
4,Getting Started with Bottlerocket,https://explore.skillbuilder.aws/learn/course/...,30m 00s,5.0,Free,Fundamental,"Bottlerocket is a Linux-based, open-source ope..."


In [50]:
df['Platform']='AWS'

In [35]:
course_list = pd.read_csv('explore.csv')
title_list = np.array(course_list['Course Name'])
url_list = np.array(course_list['Course URL'])
cost_list = np.array(course_list['Cost'])
duration_list = np.array(course_list['Duration'])
rating_list=np.array(course_list['Rating'])

In [51]:
df.to_csv('AWS.csv', index=False, encoding='utf_8_sig')